# VCM Grab n Focus Ctrl

In [1]:
import numpy as np
import subprocess
import glob
import skimage.io as imio
import re
from skimage.color import rgb2gray
import skimage
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
## ip good for the period of quarantine
IP = '10.236.176.152'
SID = 0

In [3]:
## echo + sudo -S is to bypass the pw
## good for the purple module with 2 VCMs mounted

def initialFocus(sid=SID, ip = IP):
    cmD='ssh nvidia@'+ip+' "echo "nvidia" | sudo -S ~/Documents/cam_on_Tegra/testCamera -id '+str(sid)+' -init "'
    pop=subprocess.run(cmD, stdout=subprocess.PIPE, shell=True)
    return pop.stdout
def moveFocus(sid=SID, pos=512, ip = IP):
    cmD='ssh nvidia@'+ip+' "echo "nvidia" | sudo -S ~/Documents/cam_on_Tegra/testCamera -id '+str(sid)+' -mod '+str(pos)+'"'
    pop=subprocess.run(cmD, stdout=subprocess.PIPE, shell=True)
    return pop.stdout
def setFocus(sid=SID, pos=512, ip = IP):
    cmD='ssh nvidia@'+ip+' "echo "nvidia" | sudo -S ~/Documents/cam_on_Tegra/testCamera -id '+str(sid)+' -set '+str(pos)+'"'
    pop=subprocess.run(cmD, stdout=subprocess.PIPE, shell=True)
    return pop.stdout

def grab_image(sid=SID, imres=8, ip = IP):

    subprocess.run('ssh nvidia@'+ip+' "rm testGrab*"', \
                   stdout=subprocess.PIPE, shell=True)
    subprocess.run('rm testGrab*', \
                    stdout=subprocess.PIPE, shell=True)

    subprocess.run('ssh nvidia@'+ip+' "nvgstcapture-1.0 -m 1 --sensor-id '+str(sid)+' -A --capture-auto 1 \
                    --file-name testGrab --image-res '+str(imres)+'"', stdout=subprocess.PIPE, shell=True)
    subprocess.run('scp nvidia@'+ip+':testGrab* .', \
                   stdout=subprocess.PIPE, shell=True)
    pop=subprocess.run('ls',stdout=subprocess.PIPE, shell=True)
    rePop=pop.stdout
    rePop=rePop.decode("utf-8")
    fileName = re.search(r'testGrab(.*).jpg', rePop)
    fileName=fileName.group()
    pop=imio.imread(fileName)
    pop=skimage.transform.rotate(pop,180)
    
    return pop

In [4]:
def grab_raw(sid=SID, imres=8, cameraIP=IP, expoTime = 0.1):
    # I will use the file name qhGrab.... for the image grabbed from the camera. First step is to get rid of previously grabbed images
    cmdA='ssh nvidia@'+cameraIP
    cmdB='scp nvidia@'+cameraIP
    subprocess.run('rm qhGrab*', \
                   stdout=subprocess.PIPE, shell=True)
    subprocess.run('rm *raw', \
                   stdout=subprocess.PIPE, shell=True)
    # ok, now let's grab an image on the tegra
    cmD=cmdA+' "nvgstcapture-1.0 -m 1 --sensor-id '+str(sid)+' -A --capture-auto 1 --dump-bayer\
 --file-name qhGrab --aeLock --exposure-time='+str(expoTime)+ ' --image-res '+ str(imres)+'"'
    subprocess.run(cmD, stdout=subprocess.PIPE, shell=True)
    tCmd=cmdB+':qhGrab* .'
    subprocess.run(cmdB+':qhGrab* .', \
                   stdout=subprocess.PIPE, shell=True)
    subprocess.run(cmdB+':*.raw .', \
                   stdout=subprocess.PIPE, shell=True)
    #ok, now let's find the file name of the image we just grabbed and transfered 
    pop=subprocess.run('ls',stdout=subprocess.PIPE, shell=True)
    rePop=pop.stdout
    rePop=rePop.decode("utf-8")
    fileName = re.search(r'qhGrab(.*).jpg', rePop)
    fileName=fileName.group()
    jIm=imio.imread(fileName)
    jIm=skimage.transform.rotate(jIm,180)
    fileName = re.search(r'nvcamtest_bayer(.*).raw', rePop)
    fileName=fileName.group()

    rIm=np.fromfile(fileName,dtype=np.dtype('i2'))
    # Note the array dtype above of >i2. The > means ‘big-endian’ (< is little-endian)
    # and i2 means ‘signed 2-byte integer’. For example, if our data represented a 
    # single unsigned 4-byte little-endian integer, the dtype string would be < u4 .
    rows=2160
    runL=3840
    rIm=np.reshape(rIm[:rows*runL],(rows,runL))
    
    subprocess.run(cmdA+' "mv qhGrab* ~/Desktop/"', \
                   stdout=subprocess.PIPE, shell=True)
    subprocess.run(cmdA+' "mv *.raw ~/Desktop/"', \
                   stdout=subprocess.PIPE, shell=True)
    return jIm, rIm

In [5]:
# initialFocus()

In [6]:
curr = 512
setFocus(pos = curr)

for expo_time in np.logspace(-4.0, -1.0, num=4):
    jIm, rIm = grab_raw(expoTime = expo_time)
# # gray_img = rgb2gray(img)

# plt.figure(figsize=(20, 16))
# plt.imshow(jIm)
# plt.show()

In [7]:
# fig, ax = plt.subplots(1, 2, figsize=(20, 16))
# ax[0].imshow(jIm)
# ax[1].imshow(rIm)
# fig.show()